In [1]:
import pandas as pd


#  Answer of Questions 4-5

In [2]:
df = pd.read_csv('country_vaccination_stats.csv')


min_vaccinations = df.groupby('country')['daily_vaccinations'].min().reset_index()  #creating a new df with min values of each country


df['daily_vaccinations'].fillna(0, inplace=True)  #replacing all of the Nan values with zeros

min_vaccinations.fillna(0,inplace=True)

df.head()  
df = pd.merge(df, min_vaccinations, on='country', suffixes=('', '_min'))  #merging new df with old one on country column


for index, row in df.iterrows():   #using iterrows for having a boolean, if a column equal to 0 it will make it equal to new columns value
    if row['daily_vaccinations'] == 0:
        df.loc[index, 'daily_vaccinations'] = row['daily_vaccinations_min']
        

df.drop('daily_vaccinations_min', axis=1, inplace=True)  #lastly dropping this new dummy column

In [3]:
df.isnull().sum()

country               0
date                  0
daily_vaccinations    0
vaccines              0
dtype: int64

In [4]:
df

,country,date,daily_vaccinations,vaccines
0,Argentina,12/29/2020,6483.0,Sputnik V
1,Argentina,12/30/2020,15656.0,Sputnik V
2,Argentina,12/31/2020,15656.0,Sputnik V
3,Argentina,1/1/2021,11070.0,Sputnik V
4,Argentina,1/2/2021,8776.0,Sputnik V
...,...,...,...,...
1497,Wales,1/20/2021,11105.0,"Oxford/AstraZeneca, Pfizer/BioNTech"
1498,Wales,1/21/2021,12318.0,"Oxford/AstraZeneca, Pfizer/BioNTech"
1499,Wales,1/22/2021,15148.0,"Oxford/AstraZeneca, Pfizer/BioNTech"
1500,Wales,1/23/2021,17371.0,"Oxford/AstraZeneca, Pfizer/BioNTech"


#  Answer of Question 6

In [5]:
df.groupby("country").agg({"daily_vaccinations":"median"}).sort_values(by="daily_vaccinations",ascending=False).head(3)

,daily_vaccinations
country,
United States,378253.0
China,276786.0
India,173922.0


#  Answer of Question 7

In [6]:
df[df["date"]=="1/6/2021"]["daily_vaccinations"].sum()

1485255.0

#  Answer of Question 8

UPDATE country_vaccination_stats AS t
JOIN (
    SELECT 
        country, 
        COALESCE(
            (SELECT daily_vaccinations FROM country_vaccination_stats WHERE country = t1.country AND daily_vaccinations IS NOT NULL ORDER BY daily_vaccinations LIMIT 1), 
            0
        ) AS median_daily_vaccinations
    FROM country_vaccination_stats t1
    WHERE daily_vaccinations IS NULL
    GROUP BY country
) t2 ON t.country = t2.country
SET t.daily_vaccinations = t2.median_daily_vaccinations;